In [ ]:
import math

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# 读取CSV文件

df_k = pd.read_csv('kz_1.csv')
df_theta = pd.read_csv('Theta_1.csv')

# 创建空列表存储计算结果
results = []

# 获取数据的行数和列数
num_rows = df_k.shape[0]
num_cols = df_k.shape[1]

# 创建空数组存储计算结果
results = np.zeros((num_rows, num_cols))

# 遍历每个数据点，计算结果
for i in range(num_rows):
    for j in range(num_cols):
        k_i = df_k.iloc[i, j]
        theta_i = df_theta.iloc[i, j]
        results[i, j] = -1 / k_i * (theta_i + 2 * math.pi * 0)

# 绘制色度图
plt.imshow(results, cmap='viridis', aspect='auto')
plt.colorbar()
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.title('Color Map of h values')
plt.show()
